In [7]:
import tensorflow as tf
from keras import models, layers
import matplotlib.pyplot as plt

In [8]:
IMAGE_SIZE = 256
BATCH_SIZE = 32

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "E:\Python\Data Science\Potato Disease Classification\Data\PlantVillage",
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

NotFoundError: Could not find directory \Potato Disease Classification\Data\PlantVillage